In [1]:
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

In [2]:
# Load the model
model = load_model('./modeldata/pen_classification_keras_model.h5')

In [3]:
Object_labels = {}
lines = []
with open("./modeldata/labels.txt") as f:
    lines = f.readlines()

for line in lines:
    line = line.strip()
    labels_substrings = line.split(" ")
    myStringLabel = "".join(labels_substrings[1:])
    Object_labels[int(labels_substrings[0])] = myStringLabel
    # print(labels_substrings, myStringLabel)

print(Object_labels)

{0: 'BluePen', 1: 'NoPen', 2: 'TVStand', 3: 'OrangePen', 4: 'GreenPen'}


In [4]:
# create batch 1 empty placeholder
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [5]:
import glob
greenpens_imgpaths = glob.glob("./input/greenpen/*.jpg")

In [6]:
preds_distribution = {}
for green_img_path in greenpens_imgpaths:
    image = Image.open(green_img_path)
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
    # Load the image into the array
    data[0] = normalized_image_array

    # run the inference
    prediction = model.predict(data)
    pred_max_index = np.argmax(prediction,axis=-1)[0]
    Pred_label = Object_labels[pred_max_index]
    PredConfidence = prediction[0][pred_max_index]

    # print(f"path: {green_img_path} prediction: {Pred_label} Confi: {PredConfidence}")

    if Pred_label in preds_distribution:
        preds_distribution[Pred_label]+=1
    else:
        preds_distribution[Pred_label]=1


print(f"Total green images: {len(greenpens_imgpaths)}\n\nPred Dist: {preds_distribution} ")

Total green images: 105

Pred Dist: {'TVStand': 11, 'GreenPen': 78, 'BluePen': 2, 'OrangePen': 14} 
